In [ ]:
import subprocess
import requests
import re
import os
from pathlib import Path
from zipfile import ZipFile
from PIL import Image
import site
import json

In [ ]:
working_folder = Path('.').resolve()
dotnet_type = 'runtime'
package_path = Path(site.getusersitepackages())
kernel_path = None
result = subprocess.run(["jupyter", "kernelspec", "list"], capture_output=True, text=True)
for line in result.stdout.splitlines():
    if 'python3' in line:
        parts = re.split(r'\s+', line.strip())
        kernel_path = Path(parts[1]).parent
        break
print(f"Package path: {package_path}")
print(f"Kernel path: {kernel_path}")

# Install .Net

In [ ]:
if dotnet_type == 'sdk':
    base_url = 'https://dotnet.microsoft.com/download'
    page = requests.get(base_url)
    hrefs = re.findall(r'href="([^"]+)"', page.text)
    links = [h for h in hrefs if 'sdk' in h and 'windows-x64-installer' in h]
    versions = [re.search(r'sdk-((\d+\.){1,2}\d+)-', l).group(1) for l in links if re.search(r'sdk-((\d+\.){1,2}\d+)-', l)]
else:
    base_url = 'https://dotnet.microsoft.com/en-us/download/dotnet/9.0/runtime'
    page = requests.get(base_url)
    hrefs = re.findall(r'href="([^"]+)"', page.text)
    links = [h for h in hrefs if 'runtime' in h and 'windows-x64-installer' in h]
    versions = [re.search(r'runtime-((\d+\.){1,2}\d+)-', l).group(1) for l in links if re.search(r'runtime-((\d+\.){1,2}\d+)-', l)]

latest_ver = sorted(versions, key=lambda s: [int(x) for x in s.split('.')], reverse=True)[0]
latest_uri = None
for l in links:
    if f"{dotnet_type}-{latest_ver}-windows-x64-installer" in l:
        latest_uri = 'https://dotnet.microsoft.com' + l
        break

installer_page = requests.get(latest_uri)
installer_hrefs = re.findall(r'href="([^"]+\.exe)"', installer_page.text)
file_uri = installer_hrefs[0]
exe_path = working_folder / 'dotnet.exe'
with requests.get(file_uri, stream=True) as r:
    r.raise_for_status()
    with open(exe_path, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)

In [ ]:
subprocess.run([str(exe_path), '/install', '/passive', '/norestart'], check=True)

In [ ]:
exe_path.unlink()

# Install PowerShell Kernel

In [ ]:
release_uri = 'https://github.com/sakaztk/Jupyter-PowerShellSDK/releases'
latest_release_json = requests.get(f"{release_uri}/latest", headers={'Accept':'application/json'}).json()
version_string = re.search(r'.*tag/(.*)', latest_release_json['update_url']).group(1)
assets_page = requests.get(f"{release_uri}/expanded_assets/{version_string}").text
asset_links = re.findall(r'href="([^"]+PowerShell5\.zip)"', assets_page)
file_uri = 'https://github.com' + asset_links[0]
zip_path = working_folder / 'PowerShell5.zip'
with requests.get(file_uri, stream=True) as r:
    r.raise_for_status()
    with open(zip_path, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)

In [ ]:
install_path = package_path / 'powershell5_kernel'
install_path.mkdir(parents=True, exist_ok=True)
with ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(install_path)
zip_path.unlink()

kernel_dir = kernel_path / 'powershell5'
kernel_dir.mkdir(parents=True, exist_ok=True)
logo_url = 'https://raw.githubusercontent.com/PowerShell/PowerShell/master/assets/Powershell_64.png'
logo_64_path = kernel_dir / 'logo-64x64.png'
with requests.get(logo_url, stream=True) as r:
    r.raise_for_status()
    with open(logo_64_path, 'wb') as f:
        for chunk in r.iter_content(8192):
            f.write(chunk)

logo_32_path = kernel_dir / 'logo-32x32.png'
img = Image.open(logo_64_path)
img32 = img.resize((32,32))
img32.save(logo_32_path)
kernel_json = {
    "argv": [str(install_path / "Jupyter_PowerShell5.exe"), "{connection_file}"],
    "display_name": "PowerShell 5",
    "language": "Powershell"
}
with open(kernel_dir / 'kernel.json', 'w', encoding='utf-8') as f:
    json.dump(kernel_json, f, indent=4)

for png in install_path.glob('*.png'):
    png.rename(kernel_dir / png.name)